In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Load training data
df = pd.read_csv("/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/train.csv")

# Define feature sets
base_fractions = ['Component1_fraction', 'Component2_fraction', 'Component3_fraction', 'Component4_fraction']  # Removed Component5_fraction (low variance)
property_features = lambda i: [f'Component{j}_Property{i}' for j in range(1, 6)]

# Define the function to get the best model for each target
def get_trained_final_model(data, target, property_name):
    final_model_info = {
        'BlendProperty1': ('GPR', make_pipeline(StandardScaler(), GaussianProcessRegressor(kernel=C(1.0) * RBF(2.0), n_restarts_optimizer=5, random_state=42))),
        'BlendProperty2': ('GPR', make_pipeline(StandardScaler(), GaussianProcessRegressor(kernel=C(1.0) * RBF(2.0), n_restarts_optimizer=5, random_state=42))),
        'BlendProperty3': ('ElasticNet', ElasticNet(alpha=0.1, l1_ratio=0.7, random_state=42)),
        'BlendProperty4': ('RandomForest', RandomForestRegressor(n_estimators=150, max_depth=12, random_state=42)),
        'BlendProperty5': ('SVR', make_pipeline(StandardScaler(), SVR(C=3.0, epsilon=0.1))),
        'BlendProperty6': ('ElasticNet', ElasticNet(alpha=0.05, l1_ratio=0.3, random_state=42)),
        'BlendProperty7': ('SVR_poly', make_pipeline(StandardScaler(), SVR(kernel='poly', degree=2, C=2.0))),
        'BlendProperty8': ('ElasticNet', ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)),
        'BlendProperty9': ('GPR', make_pipeline(StandardScaler(), GaussianProcessRegressor(kernel=C(1.0) * RBF(2.0), n_restarts_optimizer=3, random_state=42))),
        'BlendProperty10': ('NN', Sequential([
            Dense(64, activation='relu', input_shape=(data.shape[1],)),
            Dropout(0.3),
            Dense(64, activation='relu'),
            Dense(1)
        ]))
    }

    model_name, model = final_model_info[property_name]
    X = data
    y = target

    print(f"📌 Training {model_name} for {property_name}...")

    if model_name == 'NN':
        model.compile(optimizer='adam', loss='mae')
        model.fit(X, y, epochs=150, batch_size=32, verbose=0)
    elif isinstance(model, Pipeline):
        model.fit(X, y)
    else:
        model.fit(X, y)

    return model

# Prepare test data
test_df = pd.read_csv("/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/test.csv")
submission_df = pd.read_csv("/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/sample_solution.csv")
test_ids = test_df['ID']
test_df_features = test_df.drop(columns=['ID'])

# Generate predictions
for i in range(1, 11):
    prop = f"BlendProperty{i}"
    features = base_fractions + property_features(i)
    X_train = df[features]
    y_train = df[prop]
    X_test = test_df_features[features]

    model = get_trained_final_model(X_train, y_train, prop)
    preds = model.predict(X_test)

    submission_df[prop] = preds

# Save to CSV
submission_df.to_csv("finbath_submission.csv", index=False)
print("\n✅ Final submission file 'final_model_submission.csv' saved successfully.")


/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


📌 Training GPR for BlendProperty1...


/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/sklearn/gaussian_process/_gpr.py:663: ConvergenceWarning: lbfgs failed to converge after 10 iteration(s) (status=2):
ABNORMAL: 

You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/sklearn/gaussian_process/_gpr.py:663: ConvergenceWarning: lbfgs failed to converge after 0 iteration(s) (status=2):
ABNORMAL: 

You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object

📌 Training GPR for BlendProperty2...


/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

📌 Training ElasticNet for BlendProperty6...
📌 Training SVR_poly for BlendProperty7...


/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


📌 Training ElasticNet for BlendProperty8...
📌 Training GPR for BlendProperty9...


/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


📌 Training NN for BlendProperty10...
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 

✅ Final submission file 'final_model_submission.csv' saved successfully.
